In [3]:
import pandas as pd

In [4]:
df = pd.read_csv(r"C:\Users\SUKI\OneDrive\Desktop\sg-resale-flat-prices-2017-onwards.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181262 entries, 0 to 181261
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                181262 non-null  object 
 1   town                 181262 non-null  object 
 2   flat_type            181262 non-null  object 
 3   block                181262 non-null  object 
 4   street_name          181262 non-null  object 
 5   storey_range         181262 non-null  object 
 6   floor_area_sqm       181262 non-null  float64
 7   flat_model           181262 non-null  object 
 8   lease_commence_date  181262 non-null  int64  
 9   remaining_lease      181262 non-null  object 
 10  resale_price         181262 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 15.2+ MB


In [6]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [7]:
df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
remaining_lease        0
resale_price           0
dtype: int64

In [8]:
## changing Datatype for column year
df['date'] = pd.to_datetime(df['month'], format='%Y-%m')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181262 entries, 0 to 181261
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   month                181262 non-null  int32         
 1   town                 181262 non-null  object        
 2   flat_type            181262 non-null  object        
 3   block                181262 non-null  object        
 4   street_name          181262 non-null  object        
 5   storey_range         181262 non-null  object        
 6   floor_area_sqm       181262 non-null  float64       
 7   flat_model           181262 non-null  object        
 8   lease_commence_date  181262 non-null  int64         
 9   remaining_lease      181262 non-null  object        
 10  resale_price         181262 non-null  float64       
 11  date                 181262 non-null  datetime64[ns]
 12  year                 181262 non-null  int32         
dtypes: datetime64[

In [10]:
## New column to find flat age
df['flat_age'] = df['year'] - df['lease_commence_date']

In [11]:
## converting floor_area_sqm to square feet
conversion_factor = 10.76391
df['sqft'] = (df['floor_area_sqm'] * conversion_factor).round(0)

In [12]:
## Calculating price per sqft
df['price_per_sqft'] = (df['resale_price']/df['sqft']).round(0)

In [13]:
## Flat condition and floor rise columns using quantile cuts 
floors = df['storey_range'].str.split(' TO ',expand = True).astype(int)
floor_avg = floors.mean(axis=1)
df['floor_rise'] = pd.cut(floor_avg,
                          bins=[0,5,10,float('inf')],
                          labels=['Low rise','Mid rise','High rise']
                         )
df['flat_condition'] = pd.qcut(
    df['flat_age'],
    q=4,
    labels=['Very New', 'New', 'Mid-age', 'Old']
)


In [14]:
## region mapping for each town
region_map = {
    'CENTRAL': ['CENTRAL AREA', 'BUKIT MERAH', 'QUEENSTOWN', 'BUKIT TIMAH',
        'TOA PAYOH', 'KALLANG/WHAMPOA', 'BISHAN'],
    'EAST': [ 'BEDOK', 'TAMPINES', 'PASIR RIS', 'GEYLANG','MARINE PARADE'],
    'WEST': [ 'JURONG EAST', 'JURONG WEST', 'CLEMENTI',
        'BUKIT BATOK', 'BUKIT PANJANG', 'CHOA CHU KANG'],
    'NORTH': ['WOODLANDS', 'YISHUN', 'SEMBAWANG', 'SUNGEI KADUT'],
    'NORTH-EAST': ['HOUGANG', 'SENGKANG', 'PUNGGOL',
        'SERANGOON', 'ANG MO KIO']
             }
town_to_region = {
                 town: region
                 for region, towns in region_map.items()
                 for town in towns
                }
df['region'] = df['town'].map(town_to_region)
df['region'].value_counts()
df[df['region'].isna()]['town'].unique()
df['region'] = df['region'].astype('category')


In [15]:
df.columns
df_final = df.drop(columns=['month','block','street_name','lease_commence_date','storey_range','remaining_lease','floor_area_sqm'])

In [16]:
df_final.head()

,town,flat_type,flat_model,resale_price,date,year,flat_age,sqft,price_per_sqft,floor_rise,flat_condition,region
0,ANG MO KIO,2 ROOM,Improved,232000.0,2017-01-01,2017,38,474.0,489.0,High rise,Old,NORTH-EAST
1,ANG MO KIO,3 ROOM,New Generation,250000.0,2017-01-01,2017,39,721.0,347.0,Low rise,Old,NORTH-EAST
2,ANG MO KIO,3 ROOM,New Generation,262000.0,2017-01-01,2017,37,721.0,363.0,Low rise,Old,NORTH-EAST
3,ANG MO KIO,3 ROOM,New Generation,265000.0,2017-01-01,2017,37,732.0,362.0,Low rise,Old,NORTH-EAST
4,ANG MO KIO,3 ROOM,New Generation,265000.0,2017-01-01,2017,37,721.0,368.0,Low rise,Old,NORTH-EAST


In [17]:
!pip install psycopg2-binary sqlalchemy

## Connecting Python script to PostgreSQL

In [18]:
from sqlalchemy import create_engine
username = "postgres"
password = "Db123!"
host = "localhost"
port = "5432"
database = "resale_hdb"
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")
table_name = "hdbprice"
df_final.to_sql(table_name,engine,if_exists="replace",index=False)
print(f"Data successfully loaded into table '{table_name}' in database '{database}'")

Data successfully loaded into table 'hdbprice' in database 'resale_hdb'


In [19]:
df_final.to_csv('hdb_data.csv')

In [20]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181262 entries, 0 to 181261
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   town            181262 non-null  object        
 1   flat_type       181262 non-null  object        
 2   flat_model      181262 non-null  object        
 3   resale_price    181262 non-null  float64       
 4   date            181262 non-null  datetime64[ns]
 5   year            181262 non-null  int32         
 6   flat_age        181262 non-null  int64         
 7   sqft            181262 non-null  float64       
 8   price_per_sqft  181262 non-null  float64       
 9   floor_rise      181262 non-null  category      
 10  flat_condition  181262 non-null  category      
 11  region          181262 non-null  category      
dtypes: category(3), datetime64[ns](1), float64(3), int32(1), int64(1), object(3)
memory usage: 12.3+ MB
